In [1]:
# packages
import array as parr
import math
from typing import List, Tuple, Optional, Union, Callable
from random import choice, uniform
from abc import ABC, abstractmethod
from copy import deepcopy

In [160]:
class Arr(ABC):
    def __init__(self, arg):
        self.arg = arg
    
    @abstractmethod
    def scalar_op(self, other, fn):
        raise NotImplementedError
        
    @abstractmethod
    def mk_data(self, arg):
        raise NotImplementedError
        
class Vector(Arr):
    def __init__(self, arg: Union[float, List[float], parr.array],
                size: Optional[int]=None):
        self._size = size
        super().__init__(arg)
        self._data = None
        
    def from_float(self, f: float):
        if self._size is None:
            raise ValueError("size can not be not if using scalar value")
        self._data = [f for i in range(self._size)]
        
    def from_list(self, lst: List[float]):
        self._data = [float(f) for f in lst]
        self._size = len(self.data)
        
    def from_array(self, arr: parr.array):
        self._data = [float(f) for f in arr.tolist()]
        self._size = len(self.data)
        
    def mk_data(self, arg):
        if isinstance(arg, (int, float)):
            self.from_float(arg)
        elif isinstance(arg, list):
            if arg:
                if isinstance(arg[0], (int, float)):
                    self.from_list(arg)
        elif isinstance(arg, parr.array):
            self.from_array(arg)
        else:
            raise TypeError("argument must be float or list")
            
    @property
    def data(self):
        if self._data is None:
            self.mk_data(arg=self.arg)
        return self._data
    
    @property
    def size(self):
        if self._size is None:
            self.mk_data(arg=self.arg)
        return self._size
    
    def __str__(self):
        "string representation"
        data = [arr for arr in self.data]
        return str(data)
    
    def to_list(self) -> List[List[float]]:
        ""
        return deepcopy(self.data)
    
    @staticmethod
    def _to_random(size: int, mn=float(0), mx=float(250)):
        "random data"
        arr = [uniform(mn,mx) for c in range(size)]
        return Vector(arr)
    
    def to_random(self, mn=float(0), mx=float(250)):
        return self._to_random(size=self.size, mn=mn, mx=mx)
        
    @classmethod
    def zeros_like(cls, vec):
        return Vector(arg=0, size=vec.size)
            
    def copy(self):
        dt = deepcopy(self.data)
        return Vector(dt)
    
    def scalar_op(self, sval: float, fn):
        if not isinstance(sval, (float, int)):
            raise TypeError("argument value must be float")
        sval = float(sval) 
        self._data = [fn(v, sval) for v in self.data]
        return self.copy()
    
    def scalar_add(self, sval: float):
        ""
        return self.scalar_op(sval, fn=lambda x,y: x+y)
    
    def scalar_subt(self, sval: float):
        return self.scalar_op(sval, fn=lambda x,y: x-y)
        
    def scalar_mult(self, sval: float):
        return self.scalar_op(sval, fn=lambda x,y: x * y)
        
    def scalar_div(self, sval: float):
        if sval != 0:
            return self.scalar_op(sval, fn=lambda x,y: x / y)
        else:
            raise ZeroDivisionError("argument can not be 0")
            
    def vector_op(self, vals: List[float], fn: Callable[[float], float]):
        ""
        if len(self.data) != len(vals):
            raise ValueError("Arguments can not be added to rows, "+
                             "do not have the same shape")
        data = []
        for i in range(len(self.data)):
            val = self.data[i]
            v = vals[i]
            data.append(fn(val, v))
        self._data = data
        return self.copy()
    
    def vector_add(self, vals: List[float]):
        ""
        return self.vector_op(vals, fn=lambda x,y: x+y)
    
    def vector_subt(self, vals: List[float]):
        return self.vector_op(vals, fn=lambda x,y: x-y)
    
    def vector_mult(self, vals: List[float]):
        return self.vector_op(vals, fn=lambda x,y: x*y)
    
    def vector_div(self, vals: List[float]):
        if any([v==0 for v in vals]):
            raise ZeroDivisionError("vector contains zero")
        return self.vector_op(vals, fn=lambda x,y: x/y)

    def typed_op(self, other, 
                 scalar_op: Callable[[float], Arr], 
                 vec_op: Callable[[List[float]], Arr]
                ):
        if isinstance(other, (float, int)):
            return scalar_op(other)
        elif isinstance(other, list):
            if isinstance(other[0], (float, int)):
                return vec_op(other)
        elif isinstance(other, Vector):
            return vec_op(other.to_list())
        else:
            raise ValueError(
                "argument is either a scalar (float, int) or a vector (List[float])"
            )
    
    def __add__(self, other):
        return self.typed_op(other=other,
                            scalar_op=self.scalar_add,
                            vec_op=self.vector_add)

    def __sub__(self, other):
        return self.typed_op(other=other,
                            scalar_op=self.scalar_subt,
                            vec_op=self.vector_subt)
    
    def __truediv__(self, other):
        return self.typed_op(other=other,
                            scalar_op=self.scalar_div,
                            vec_op=self.vector_div)
    
    def __mul__(self, other):
        return self.typed_op(other=other,
                            scalar_op=self.scalar_mult,
                            vector_op=self.vector_mult)
            
    def __contains__(self, other):
        if not isinstance(other, (float, int)):
            return False
        else:
            for val in self.data:
                if col == other:
                    return True
            return False
        
    def __getitem__(self, key):
        if isinstance(key, int):
            if key < 0:
                raise IndexError("index can not be smaller than 0")
            elif key >= self.size:
                print(key)
                print(self.size)
                raise IndexError("index can not be bigger than row size")
            else:
                return self.data[key]
        else:
            raise TypeError("Argument type must be int")
            
    def __setitem__(self, key, val):
        """        
        """
        if isinstance(key, int):
            if key < 0:
                raise IndexError("index can not be smaller than 0")
            elif key >= self.size:
                raise IndexError("index can not be bigger than row size")
            if not isinstance(val, (int, float)):
                raise TypeError("value must be of int or float type")
            self.data[key] = val
        else:
            raise TypeError("key must either be tuple with 2 elements or an int")
            
    def __round__(self, k: Optional[int] = None):
        ""
        rdata = deepcopy(self.data)
        roundfn = lambda x: round(x, k) if k is not None else lambda x: round(x)
        data = []
        for val in rdata:
            data.append(roundfn(val))
        return Vector(data)
    
    def __len__(self):
        return len(self.data)

In [161]:
class MatArr(Arr):
    def __init__(self, arg: Union[float, 
                                  List[float], 
                                  List[parr.array],
                                  List[List[Union[float, int]]],
                                  List[Vector]
                                 ], 
                 size: Optional[Tuple[int, int]]=None):
        self._size = size
        super().__init__(arg)
        self._data = None
        
    def from_list_of_vectors(self, arg):
        ""
        row_size, column_size = len(arg), arg[0].size
        self._size = (row_size, column_size)
        data = []
        for vector in arg:
            if vector.size != column_size:
                raise ValueError("vector size should be same for all vectors")
            data.append(vector)
        self._data = data
        
    def from_list_of_list(self, arg):
        "make matrix from list of list"
        narg = [Vector(a) for a in arg]
        self.from_list_of_vectors(narg)
     
    def from_float(self, f: float):
        if self._size is None:
            raise ValueError("size can not be not if using scalar value")
        row_size, column_size = self._size
        data = []
        for r in range(row_size):
            arr = Vector(f, size=column_size)
            data.append(arr)
        self.from_list_of_vectors(data)
        
    def from_list(self, lst: List[float], axis=0):
        "make matrix from list"
        if not isinstance(lst[0], (float, int)):
            raise ValueError("List element must be float")
        if axis == 0:
            # make a row
            self._data = [Vector(lst)]
            self._size = (1, len(lst))
        elif axis == 1:
            # make as a column
            self._data = [Vector(float(f), size=1) for f in lst]
            self._size = (len(lst), 1)
        else:
            raise ValueError("axis must be either 0 or 1")
            
    def from_arrays(self, lst: List[parr.array]):
        ""
        self.from_list_of_vectors([Vector(arr) for arr in lst])
        
    def mk_data(self, arg):
        if isinstance(arg, float):
            self.from_float(f=arg)
        elif self.is_list_of_list(arg):
            self.from_list_of_list(arg)
        elif isinstance(arg, list):
            if isinstance(arg[0], (float, int)):
                self.from_list(lst=arg)
            elif isinstance(arg[0], parr.array):
                self.from_arrays(lst=arg)
            elif isinstance(arg[0], Vector):
                self.from_list_of_vectors(arg)
            else:
                raise ValueError("List must contain either array or float")
        else:
            raise ValueError("argument must be either list or float")
            
    @property
    def data(self):
        if self._data is None:
            self.mk_data(arg=self.arg)
        return self._data
    
    @property
    def row_size(self):
        if self._size is None:
            self.mk_data(arg=self.arg)
        return self.size[0]
    
    @property
    def column_size(self):
        if self._size is None:
            self.mk_data(arg=self.arg)
        return self.size[1]
    
    @property
    def size(self):
        if self._size is None:
            self.mk_data(arg=self.arg)
        return self._size
    
    @property
    def rows(self) -> List[Vector]:
        return self.data
    
    @property
    def columns(self) -> List[Vector]:
        colsize = self.column_size
        row_size = self.row_size
        cols = []
        for col_index in range(colsize):
            column = Vector([row[col_index] for row in self.rows])
            cols.append(column)
        return cols
    
    @property
    def T(self):
        "Transpose"
        columns = deepcopy(self.columns)
        return MatArr(columns)
    
    @property
    def transpose(self):
        "transpose"
        return self.T

    @staticmethod
    def is_list_of_list(other) -> bool:
        if isinstance(other, list):
            if len(other) == 0:
                return False
            if isinstance(other[0], (list, tuple)):
                if isinstance(other[0][0], (float, int)):
                    return True
        return False
    
    def to_random(self):
        ""
        data = []
        for vec in self.rows:
            data.append(vec.to_random())
        return MatArr(data)
    
    def matrix_op(self, vals: List[List[float]], fn):
        ""
        if self.is_list_of_list(vals):
            return self.matrix_op(MatArr(vals, size=None), fn)
        elif isinstance(vals, MatArr):
            if self.row_size != vals.row_size or self.column_size != vals.column_size:
                raise ValueError("arg size not same with this one")
        else:
            raise TypeError("arg must be either MatArr or list of lists")
            
        data = []
        for row_index in range(self.row_size):
            cols = [
                fn(self.data[row_index][col_index],
                   vals.data[row_index][col_index])
                for col_index in range(self.column_size)
                   ]
            data.append(cols)
        return MatArr(data, size=(self.row_size, self.column_size))
    
    def mat_add(self, Bmat):
        return self.matrix_op(vals=Bmat, fn=lambda x,y: x+y)
    
    def mat_subt(self, Bmat):
        return self.matrix_op(vals=Bmat, fn=lambda x,y: x-y)
    
    def mat_div(self, Bmat):
        if isinstance(Bmat, MatArr):
            if 0.0 in Bmat:
                raise ValueError("Argument matrix contains zero. Division is undefined")
            return self.matrix_op(vals=Bmat, fn=lambda x,y: x/y)
        elif self.is_list_of_list(Bmat):
            return self.mat_div(MatArr(Bmat, size=None))
    
    def mat_hadamard_product(self, Bmat):
        return self.matrix_op(vals=Bmat, fn=lambda x,y: x*y)

    def matmul(self, Bmat):
        "matrix multiplication"
        # A n*m, B m*p -> AB n*p
        asize = self.size
        bsize = Bmat.size
        result_row = asize[0]
        result_col = bsize[1]
        if asize[1] != bsize[0]:
            raise ValueError("argument matrix does not have the proper shape")
        result_size = (result_row, result_col)
        resultmat = [[0 for i in range(result_col)] for j in range(result_row)]
        Bdata = Bmat.data
        Adata = self.data
        for a in range(len(Adata)):
            for b_c in range(len(Bdata[0])):
                for b_r in range(len(Bdata)):
                    resultmat[a][b_c] += Adata[a][b_r] * Bdata[b_r][b_c]
        return MatArr(arg=resultmat, size=result_size)
    
    def typed_op(self, other, 
                 scalar_op: Callable[[float], Arr], 
                 vec_op: Callable[[Vector], Arr],
                 mat_op: Callable[[List[List[float]]], Arr]
                ):
        if isinstance(other, (float, int)):
            return scalar_op(other)
        elif isinstance(other, Vector):
            return vector_op(other)
        elif isinstance(other, MatArr):
            return mat_op(other)
        else:
            raise ValueError(
                "argument is either a scalar (float, int) or vector or matrix"
            )
            
    def scalar_op(self, s: Union[float, int], fn):
        ""
        data = [vec.scalar_op(s, fn) for vec in self.rows]
        return MatArr(data)
    
    def vector_op(self, v: Vector, fn):
        data = [vec.vector_op(v, fn) for vec in self.rows]
        return MatArr(data)
    
    def scalar_add(self, s):
        return self.scalar_op(s, fn=lambda x,y: x+y)
    
    def scalar_subt(self, s):
        return self.scalar_op(s, fn=lambda x,y: x-y)
    
    def scalar_div(self, s):
        return self.scalar_op(s, fn=lambda x,y: x/y)
    
    def scalar_mult(self, s):
        return self.scalar_op(s, fn=lambda x,y: x*y)
    
    def vector_add(self, s):
        return self.vector_op(s, fn=lambda x,y: x+y)
    
    def vector_subt(self, s):
        return self.vector_op(s, fn=lambda x,y: x-y)
    
    def vector_div(self, s):
        return self.vector_op(s, fn=lambda x,y: x/y)
    
    def vector_mult(self, s):
        return self.vector_op(s, fn=lambda x,y: x*y)
    
    def copy(self):
        data = []
        for vec in self.rows:
            data.append(vec.copy())
        return MatArr(data)
    
    def __add__(self, other):
        return self.typed_op(other=other,
                            scalar_op=self.scalar_add,
                            vec_op=self.vector_add,
                            mat_op=self.mat_add)

    def __sub__(self, other):
        return self.typed_op(other=other,
                            scalar_op=self.scalar_subt,
                            vec_op=self.vector_subt,
                            mat_op=self.mat_subt)
    
    def __truediv__(self, other):
        return self.typed_op(other=other,
                            scalar_op=self.scalar_div,
                            vec_op=self.vector_div,
                            mat_op=self.mat_div)
    
    def __mul__(self, other):
        return self.typed_op(other=other,
                            scalar_op=self.scalar_mult,
                            vector_op=self.vector_mult,
                            mat_op=self.mat_hadamard_product)
    
    def __matmul__(self, other):
        if isinstance(other, MatArr):
            return self.matmul(other)
        else:
            raise ValueError("argument must be a matrix")
            
    def __contains__(self, other):
        if not isinstance(other, (float, int)):
            return False
        else:
            for row in self.data:
                for col in row:
                    if col == other:
                        return True
            return False
        
    def __getitem__(self, key):
        if isinstance(key, int):
            if key < 0:
                raise IndexError("index can not be smaller than 0")
            if self.row_size == 1:
                return self.data[0][key]
            elif key >= self.row_size:
                raise IndexError("index can not be bigger than row size")
        elif isinstance(key, (list, tuple)):
            if len(key) != 2:
                raise IndexError("can not have indices for more than two axes")
            row, col = key
            if ((row < 0 or row >= self.row_size) or ( col < 0 or col >= self.column_size)):
                raise IndexError("given indices don't fit to current matrix")
            return self.data[row][col]
        else:
            raise TypeError("key must either be tuple with 2 elements or an int")
            
    def set_to_row(self, row_index, values: Vector):
        if len(values) != self.row_size:
            raise ValueError("value size must be equal to row size")
        if row_index < 0 or row_index >= self.row_size:
            raise IndexError("given row index is out of bounds for current matrix")
        self.data[row_index] = values
        
    def set_to_column(self, col_index, values: List[float]):
        "set values to column using column index"
        if len(values) != self.column_size:
            raise ValueError("value size must be equal to column size")
        if col_index < 0 or col_index >= self.column_size:
            raise ValueError("given column index is out of bounds for current matrix")
        for row_index in range(self.row_size):
            self.data[row_index][col_index] = values[row_index]
            
    def set_column_or_row(self, axis: Union[int, str], index: int, values: List[float]):
        if isinstance(axis, int):
            if axis == 0:
                self.set_to_row(index, values)
            elif axis == 1:
                self.set_to_column(index, values)
            else:
                raise IndexError("axis must be 0 or 1")
        elif isinstance(axis, str):
            alow = axis.lower()
            if alow == "r" or alow == "row":
                self.set_to_row(index, values)
            elif alow == "c" or alow == "column":
                self.set_to_column(index, values)
                
    def __setitem__(self, key, val):
        """
        set given val to key
        
        Ex:
        matarr = MatArr([[1,2], [3,4]])
        
        set a column:
        matarr[(1,0)] = [5, 6]
        matarr
        [[5,2], [6,4]]
        
        set a row:
        matarr[(0, 0)] = [5, 6]
        matarr
        [[5,6], [3,4]]
        
        """
        if isinstance(key, int):
            if key < 0:
                raise IndexError("index can not be smaller than 0")
            if self.row_size == 1:
                if isinstance(val, (float, int)):
                     self.data[0][key] = val
                else:
                    raise TypeError("set value must be int or float")
            elif key >= self.row_size:
                raise IndexError("index can not be bigger than row size and smaller than 0")
            
        elif isinstance(key, (list, tuple)):
            if len(key) != 2:
                raise IndexError("can not have indices for more than two axes")
            axis, index = key
            self.set_column_or_row(axis=axis, index=index, values=val)
        else:
            raise TypeError("key must either be tuple with 2 elements or an int")
            
    def __round__(self, k: Optional[int] = None):
        ""
        rdata = deepcopy(self.data)
        roundfn = lambda x: round(x, k) if k is not None else lambda x: round(x)
        data = []
        for row in rdata:
            cols = [roundfn(v) for v in row]
            data.append(cols)
        return MatArr(data)
    
    def __str__(self):
        s = "[\n"
        for vec in self.rows:
            vstr = "["
            for v in vec.data:
                vstr += str(v) + " "
            vstr += "]\n"
            s += vstr
        s += "]"
        return s

In [162]:
matarr = MatArr(arg=1.5, size=(6,3))

In [163]:
print(matarr)

[
[1.5 1.5 1.5 ]
[1.5 1.5 1.5 ]
[1.5 1.5 1.5 ]
[1.5 1.5 1.5 ]
[1.5 1.5 1.5 ]
[1.5 1.5 1.5 ]
]


In [164]:
matarr = MatArr(arg=[1.6, 8.3, 3.76], size=None)
print(matarr)

[
[1.6 8.3 3.76 ]
]


In [165]:
matarr.to_random()
print(matarr)

[
[1.6 8.3 3.76 ]
]


In [166]:
matarr = MatArr(arg=[1.6, 8.3, 3.76], size=None)
print(matarr + 6)

[
[7.6 14.3 9.76 ]
]


In [167]:
def derivative(f: Callable[[float], float], x: float, h=1e-8) -> float:
    "Takes the derivative of f"
    f_t = (f(x + h) - f(x)) / h
    return f_t

def partial_derivative(f: Callable[[Vector], float], 
                       arguments: Vector, 
                       argument_position: int, 
                       h=1e-8) -> float:
    """
    Basic implementation of partial derivative
    """
    if not isinstance(arguments, Vector):
        raise TypeError("arguments must be vector")
    arguments_copy = arguments.copy() # in order to leave original arguments untouched
    farg = f(arguments)
    if not isinstance(farg, float):
        raise TypeError("function must output a float value")
    arguments_copy[argument_position] = arguments_copy[argument_position] + h
    farg_cp = f(arguments_copy)
    return (farg_cp - farg) / h

In [168]:
# simple derivative test
def testfn(x): return x * x
d = derivative(testfn, x=5)
print("x^2 derivative = 2x => 5^2 derivative 2 * 5", 
      10 == round(d, 3))
# testing partial derivative
def test_part_fn(xs: list): return (xs[0] ** 2) + (xs[0] * xs[1]) + (xs[1] ** 2)
# x^2 + xy + y^2
# f_x' = 2x + y , f_y' = 2y + x
xd = partial_derivative(f=test_part_fn, arguments=Vector([2, 6]), argument_position=0)
yd = partial_derivative(f=test_part_fn, arguments=Vector([2, 6]), argument_position=1)
print(xd)
print(yd)
print("f(x,y) = x^2 + xy + y^2, x=2, y=6")
print("f_x'(x,y) = 2x + y => 4 + 6 = 10")
print("f_y'(x,y) = 2y + x => 2 + 12 = 14")
print("f_x'(x,y) = 10", round(xd, 3) == 10)
print("f_y'(x,y) = 14", round(yd, 3) == 14)

x^2 derivative = 2x => 5^2 derivative 2 * 5 True
10.000000116860974
14.000000447822458
f(x,y) = x^2 + xy + y^2, x=2, y=6
f_x'(x,y) = 2x + y => 4 + 6 = 10
f_y'(x,y) = 2y + x => 2 + 12 = 14
f_x'(x,y) = 10 True
f_y'(x,y) = 14 True


In [169]:
def gradient(f: Callable[[Vector], float], 
             arguments: Vector, 
             h=1e-8) -> Vector:
    """
    Basic implementation of gradient computation
    """
    partials: List[float] = []
    for argument_pos in range(arguments.size):
        partial = partial_derivative(f=f, arguments=arguments, 
                                     argument_position=argument_pos)
        partials.append(partial)
    return Vector(partials)

In [170]:
# test gradient
partials = gradient(f=test_part_fn, arguments=Vector([2, 6]))
print("∇f(x,y), gradient of f, that is [f_x', f_y'] => [10, 14]")
print("∇f(x,y) = [10, 14], ", [round(p, 3) for p in partials] == [10, 14])

∇f(x,y), gradient of f, that is [f_x', f_y'] => [10, 14]
2
2
∇f(x,y) = [10, 14],  True


In [171]:
def derivative_of_vector_function_with_known_functions(
    fs: List[Callable[[float], float]], 
    x: float,
    h=1e-8
)-> Vector:
    "Compute derivative of a vector function when member functions are known"
    derivatives = []
    for f in fs:
        d = derivative(f, x)
        derivatives.append(d)
    return Vector(derivatives)

def derivative_of_vector_function(
    f: Callable[[float], Vector], 
    x: float, 
    h=1e-8) -> Vector:
    "Derivative of a vector function where member functions are not known"
    fx = f(x)
    fx_h = f(x+h)
    fsubt = fx_h - fx
    return (fx_h - fx) / h

In [172]:
# testler
def testfn(x): return MatArr([x*x, 2 * x, x - 1])
# turevi bunun 2x, 2, 1
vk = derivative_of_vector_function_with_known_functions(fs=[lambda x: x*x, 
                                     lambda x: 2 * x, 
                                     lambda x: x -1],
                                 x=3)
print("with known functions", 
      [round(v, 3) for v in vk] == [6, 2, 1])
vnk = derivative_of_vector_function(f=testfn, x=3)
print("with unknown functions", 
      [round(v, 3) for v in vnk] == [6, 2, 1])

3
3
with known functions True
3
3
with unknown functions True


In [173]:
def jacoby_with_known_functions(fs: List[Callable[[Vector], float]], 
                                args: Vector, h=1e-8) -> MatArr:
    j_mat = []
    for f in fs:
        dmat = gradient(f=f, arguments=args.copy(), h=h)
        j_mat.append(dmat)
    return MatArr(j_mat)

def jacobian(f: Callable[[Vector], Vector],
             args: Vector, h=1e-8) -> MatArr:
    ""
    j_mat = []
    for argpos in range(args.size):
        acopy = args.copy()
        acopy[argpos] = acopy[argpos] + h
        result = (f(acopy.copy()) - f(args.copy())) / h
        j_mat.append(result)
    return MatArr(j_mat).transpose

In [174]:
# from wikipedia

def f1(arg: Vector) -> float:
    if not isinstance(arg, Vector):
        raise TypeError("argument must be vector")
    x, y = arg.data
    x2 = x*x
    return x2 * y

def f2(arg: Vector) -> float:
    if not isinstance(arg, Vector):
        raise TypeError("argument must be a vector")
    x, y = arg.data
    return 5*x + math.sin(y)
        
def testfn(arg: Vector) -> Vector:
    ""
    return Vector([f1(arg), f2(arg)])

# Jacoby = [[2xy, x*x ], [5, cos(y)]]
   
j_known = jacoby_with_known_functions(
    fs=[f1,f2], args=Vector([2,1])
)
print("jacobian with known functions: ",round(j_known, 3))
j_unknown = jacobian(f=testfn, args=Vector([2,1]))
print("jacobian with unknown function: ", round(j_unknown, 3))

2
2
2
2
jacobian with known functions:  [
[4.0 4.0 ]
[5.0 0.54 ]
]
2
2
2
2
jacobian with unknown function:  [
[4.0 4.0 ]
[5.0 0.54 ]
]


In [175]:
def chain1d(f: Callable[[float], float], g: Callable[[float], float], x: float):
    ""
    delta_g = derivative(g, x)
    g_value = g(x)
    delta_f = derivative(f, g_value)
    return delta_f * delta_g

# chain test
def f(x): return 2 * x
def g(x): return x * x
# derivative 2 * 2x = 4x =1
def testfn(x): return f(g(x))
print("We have equal derivatives, right ? ", 
      round(derivative(testfn, x=2), 3) == round(chain1d(f=f, g=g, x=2), 3))

We have equal derivatives, right ?  True


In [205]:
def chain(f: Callable[[Vector], Vector], 
          g: Callable[[Vector], Vector],
          x: Vector) -> MatArr:
    ""
    g_val = g(x)
    g_j: MatArr = jacobian(f=g, args=x)
    f_j: MatArr = jacobian(f=f, args=g_val)
    print(g_j)
    print(f_j)
    result = f_j @ g_j
    print(result)
    return result

In [206]:
# test from https://mathinsight.org/chain_rule_multivariable_examples
# g(t) = (t**3, t**4)
# f(x,y) = x**2 * y
# h(t) = f(g(t))

def gfn(vs: Vector):
    t = vs[0]
    return Vector([t**3, t**4])

def f_fn(vs: Vector):
    x, y = vs.data
    return Vector([x**2 * y], size=1)

xvec = Vector([1.0, 2.0])
fxval = chain(f=f_fn, g=gfn, x=xvec)

[
[3.0000000039720476 0.0 ]
[4.000000042303498 0.0 ]
]
[
[1.999999987845058 0.999999993922529 ]
]
[
[9.999999989472883 0.0 ]
]


In [204]:
print(fxval)

[
[9.999999989472883 0.0 ]
]
